In [1]:
import os
import sys
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.autograd import grad
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
if 'higher' not in sys.modules:
  !pip install higher
import higher as higher
print(sys.version)
print(torch.__version__)

3.7.12 (default, Jan 15 2022, 18:48:18) 
[GCC 7.5.0]
1.10.0+cu111


In [2]:
# Set random seeds
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
# Set important parameters
learning_rate = 1e-2
inner_loop_iterations = 32
outer_loop_iterations = 5
num_classes = 10
noise_size = 64 # size of noise or curriculum vector
img_size = 28 # width / height of generated image
inner_loop_batch_size = 128
outer_loop_batch_size = 128
mnist_mean = 0.1307 # for normalizing mnist images
mnist_std = 0.3081 # for normalizing mnist images
imgs_per_row = num_classes

In [3]:
# Initialize MNIST transforms
transform = transforms.Compose([
transforms.Lambda(lambda x: np.array(x)),
transforms.ToTensor(),
transforms.Normalize((mnist_mean,), (mnist_std,)),
])
# Create data splits
train = datasets.MNIST('./data', train=True, transform=transform, download=True)
train, val = torch.utils.data.random_split(train, [50000, 10000])
test = datasets.MNIST('./data', train=False, transform=transform, download=True)
print('Created train, val, and test datasets.')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Created train, val, and test datasets.


In [4]:
train_loader = torch.utils.data.DataLoader(
  train, batch_size=outer_loop_batch_size, shuffle=True, drop_last=True, num_workers=1
  , pin_memory=True,
)
val_loader = torch.utils.data.DataLoader(
  val, batch_size=outer_loop_batch_size, shuffle=True, drop_last=True, num_workers=1,
  pin_memory=True,
)
test_loader = torch.utils.data.DataLoader(
  test, batch_size=outer_loop_batch_size, shuffle=True, drop_last=True, num_workers=1,
  pin_memory=True,
)

In [5]:
class Teacher(nn.Module):
  '''
  Implements a Teacher module.
  '''
  def __init__(self):
    super().__init__()
    conv1_filters = 64
    fc1_size = 1024
    self.fc2_filters = 128
    self.fc2_width = img_size
    fc2_size = self.fc2_filters * self.fc2_width * self.fc2_width
    self.fc1 = nn.Linear(noise_size + num_classes, fc1_size)
    nn.init.kaiming_normal_(self.fc1.weight, 0.1)
    self.bn_fc1 = nn.BatchNorm1d(fc1_size, momentum=0.1)
    self.fc2 = nn.Linear(fc1_size, fc2_size)
    nn.init.kaiming_normal_(self.fc2.weight, 0.1)
    self.bn_fc2 = nn.BatchNorm2d(self.fc2_filters, momentum=0.1)
    self.conv1 = nn.Conv2d(self.fc2_filters, conv1_filters, 3, 1, padding=3 // 2)
    self.bn_conv1 = nn.BatchNorm2d(conv1_filters, momentum=0.1)
    self.conv2 = nn.Conv2d(conv1_filters, 1, 3, 1, padding=3 // 2)
    self.bn_conv2 = nn.BatchNorm2d(1, momentum=0.1)
    self.tanh = nn.Tanh()
    self.learner_optim_params = nn.Parameter(torch.tensor([0.02, 0.5]), True)
  def forward(self, x, target):
    '''
    Synthesizes a batch of training examples for the learner.
    Args:
    x (torch.tensor): shape (b, 64)
    target (torch.tensor): shape (b, 10)
    '''
    # Fully connected block 1
    x = torch.cat([x, target], dim=1) # shape (b, 64+10)
    x = self.fc1(x) # shape (b, 1024)
    x = F.leaky_relu(x, 0.1)
    x = self.bn_fc1(x)
    # Fully connected block 2
    x = self.fc2(x) # shape (b, 128*28*28)
    x = F.leaky_relu(x, 0.1)
    x = x.view( # shape (b, 128, 28, 28)
    -1, self.fc2_filters, self.fc2_width, self.fc2_width
    )
    x = self.bn_fc2(x)
    # Convolutional block 1
    x = self.conv1(x) # shape (b, 64, 28, 28)
    x = self.bn_conv1(x)
    x = F.leaky_relu(x, 0.1)
    # Convolutional block 2
    x = self.conv2(x) # shape (b, 1, 28, 28)

    x = self.bn_conv2(x)
    x = (self.tanh(x) + 1 - 2 * mnist_mean) / (2 * mnist_std)
    return x, target

In [6]:
class Learner(nn.Module):
  '''
  Implements a Learner module.
  '''
  def __init__(self, num_conv1=None, num_conv2=None):
    super().__init__()
    # Randomly select and evaluate convolutional depth
    # for evaluation/comparison in neural architecture search
    if num_conv1 is None:
      conv1_filters = np.random.randint(32, 64)
    else:
      conv1_filters = num_conv1
    if num_conv2 is None:
      conv2_filters = np.random.randint(64, 128)
    else:
      conv2_filters = num_conv2
    self.conv1 = nn.Conv2d(1, conv1_filters, 3, 1)
    self.bn1 = nn.BatchNorm2d(conv1_filters, momentum=0.1)
    self.conv2 = nn.Conv2d(conv1_filters, conv2_filters, 3, 1)
    self.bn2 = nn.BatchNorm2d(conv2_filters, momentum=0.1)
    c1_size = (img_size - 3 + 1) // 2
    c2_size = (c1_size - 3 + 1) // 2
    self.fc = nn.Linear(conv2_filters * c2_size * c2_size, num_classes)
    self.bn3 = nn.BatchNorm1d(num_classes, momentum=0.1)
    self.activation = nn.LeakyReLU(0.1)
    def forward(self, x):
      x = self.conv1(x)
      x = self.activation(x)
      x = self.bn1(x)
      x = F.max_pool2d(x, 2)
      x = self.conv2(x)
      x = self.activation(x)
      x = self.bn2(x)
      x = F.max_pool2d(x, 2)
      x = torch.flatten(x, 1)
      x = self.fc(x)
      x = self.bn3(x)
      return x

In [7]:
def generate_img(img_tensor):
  '''
  Function that renders an MNIST image.
  '''
  return torchvision.transforms.ToPILImage()(1 - ((img_tensor * mnist_std) + mnist_mean))

In [8]:
teacher = Teacher()
params_to_train = list(teacher.parameters())
# If we want to use a curriculum, we initialize the learnable parameters here
use_curriculum = True
if use_curriculum:
  curriculum = nn.Parameter(torch.randn(inner_loop_iterations, inner_loop_batch_size,
                noise_size), requires_grad=True)
  params_to_train += [curriculum]
optimizer_teacher = optim.Adam(params_to_train, lr=learning_rate)
# For each inner loop iterations, we use the same sequence of labels.
# This allows the curriculum vectors to train to stable labels
label = torch.tensor([x % num_classes for x in range(inner_loop_batch_size)])
# For the inner loop loss, we use cross entropy
loss_fn = nn.CrossEntropyLoss()
# Here we initialize iterators on the train and val datasets
train_iterator = iter(train_loader)
val_iterator = iter(val_loader)
test_iterator = iter(test_loader)

In [9]:
for it, real_data in enumerate(train_loader):
  teacher.train()
  optimizer_teacher.zero_grad()
  # We also optimize the learner learning rate and momentum with the
  # outer loop updates
  learner_lr = teacher.learner_optim_params[0]
  learner_momentum = teacher.learner_optim_params[1]
  # Here we sample a learner with random number of conv filters
  learner = Learner()
  inner_optim = optim.SGD(learner.parameters(), lr=learner_lr.item(), momentum=learner_momentum.item())
  learner.train()
  inner_losses = []
  with higher.innerloop_ctx(learner, inner_optim, override={'lr': [learner_lr], 'momentum': [learner_momentum]}) as (flearner, diffopt):
    for step in range(inner_loop_iterations):
      # Data generation
      if use_curriculum:
        z_vec = curriculum[step]
      else:
        z_vec = torch.randn(inner_loop_batch_size, noise_size)
      one_hot = F.one_hot(label, num_classes)
      # Pass input to teacher to generate synthetic images
      teacher_output, teacher_target = teacher(z_vec, one_hot)
      # ====== Show intermediate generated images ======
      if step == 0:
        print('------------------ Outer loop iteration', it + 1, '------------------')
        print('Examples 0 - 9 from beginning of inner loop:')
        background = Image.new('L', (img_size * imgs_per_row + imgs_per_row + 1,

        img_size + 2))

        for i in range(imgs_per_row): # indexes column
          background.paste(generate_img(teacher_output[i]), (i * 28 + i + 1, 1))

        display(background)
      if step == (inner_loop_iterations - 1):
        print('Examples 0 - 9 from end of inner loop:')
        background = Image.new('L', (img_size * imgs_per_row + imgs_per_row + 1, img_size + 2))

        for i in range(imgs_per_row): # indexes column
          background.paste(generate_img(teacher_output[i]), (i * 28 + i + 1, 1))

        display(background)
      # Pass teacher output to the learner
      learner_output = flearner(teacher_output)
      loss = loss_fn(learner_output, label)
      diffopt.step(loss)
      inner_losses.append(loss.item())
    correct = 0
    data, target = real_data
    output = flearner(data)
    loss = loss_fn(output, target)
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy_train = correct / target.shape[0]
    print("Inner loop losses:", inner_losses)
    print("Train accuracy:", accuracy_train)
    # Compute accuracy on validation set
    data, target = next(val_iterator)
    print
    output = flearner(data)
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(target.view_as(pred)).sum().item()
    accuracy = correct / outer_loop_batch_size
    print("Val accuracy:", accuracy)
    if (it == outer_loop_iterations - 1):
      # Compute accuracy on test set
      correct = 0
      for i, (data, target) in enumerate(test_loader):
        output = flearner(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
      accuracy = correct / (outer_loop_batch_size * len(test_loader))
      print("----------------------------------")
      print("Done training...")
      print("Final test accuracy:", accuracy)
      # Final inner loop training curve
      plt.plot(np.arange(len(inner_losses)), inner_losses)
      plt.xlabel("Inner loop iteration")
      plt.ylabel("Cross entropy loss")
      plt.show()
      break
    loss.backward()
  optimizer_teacher.step()

------------------ Outer loop iteration 1 ------------------
Examples 0 - 9 from beginning of inner loop:


NotImplementedError: ignored

In [ ]:
num_architectures = 10
best_accuracy = 0
for i in range(num_architectures):
  # Randomly sample architecture
  conv1_filters = np.random.randint(1, 64)
  conv2_filters = np.random.randint(1, 128)
  learner = Learner(conv1_filters, conv2_filters)
  inner_optim = optim.SGD(learner.parameters(), lr=learner_lr.item(), momentum=learner_momentum.item())
  learner.train()
  # For some reason if we don't use higher here, accuracy drops significantly
  with higher.innerloop_ctx(learner, inner_optim, override={'lr': [learner_lr], 'momentum': [learner_momentum]}) as (flearner, diffopt):
    for step in range(inner_loop_iterations):
      # Data generation
      if use_curriculum:
        z_vec = curriculum[step]
      else:
        z_vec = torch.randn(inner_loop_batch_size, noise_size)
    one_hot = F.one_hot(label, num_classes)
    # Pass input to teacher to generate synthetic images
    teacher_output, teacher_target = teacher(z_vec, one_hot)
    # Pass teacher output to the learner
    learner_output = flearner(teacher_output)
    loss = loss_fn(learner_output, label)
    diffopt.step(loss)
    # Compute accuracy on validation set
    correct = 0
    for val_idx, (data, target) in enumerate(val_loader, 0):
      #if (val_idx == val_iterations): break
      output = flearner(data)
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy = correct / (outer_loop_batch_size * len(val_loader))
    if (accuracy > best_accuracy):
      best_accuracy = accuracy
      filter_counts = (conv1_filters, conv2_filters)
    print("------------------------- Architecture", i + 1," -------------------------")
    print("Num conv1 filters:", conv1_filters, ", Num conv2 filters:", conv2_filters
    , ", Val accuracy:", accuracy)
    if (i == num_architectures - 1):
      correct = 0
      for test_idx, (data, target) in enumerate(test_loader, 0):
      #if (test_idx == test_iterations): break
        output = flearner(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
      accuracy = correct / (outer_loop_batch_size * len(test_loader))
      print("------------------------- Best architecture -------------------------")

      print("Num conv1 filters:", filter_counts[0], ", Num conv2 filters:", filter_counts[1], ", Test accuracy:", accuracy)

In [ ]:
imgs_per_row = num_classes
rows = inner_loop_iterations // 2 * img_size + inner_loop_iterations // 2 + 1
cols = imgs_per_row * img_size + imgs_per_row + 1
background = Image.new('L', (cols, rows))
for step in range(0, inner_loop_iterations, 2): # indexes row
  if use_curriculum:
    z_vec = curriculum[step]
  else:
    z_vec = torch.randn(inner_loop_batch_size, noise_size)
  one_hot = F.one_hot(label, num_classes)
  teacher_output, teacher_target = teacher(z_vec, one_hot)
  for i in range(imgs_per_row): # indexes column
    background.paste(generate_img(teacher_output[i]), (i * img_size + i + 1, (step // 2) * img_size + (step // 2) + 1))
display(background)